In [1]:
import ctf,time,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()

In [2]:
class UnitTests:
        
    def test_3d_purturb1(self):
        
        I = random.randint(3,5)
        J = random.randint(3,5)
        K = random.randint(3,5)
        r = 2 
        sparsity = .2
        regParam = 10
        
        ctf.random.seed(42)
        U = ctf.random.random((I,r))
        V= ctf.random.random((J,r))
        W= ctf.random.random((K,r))
    
    
        # 3rd-order tensor
        T = ctf.tensor((I,J,K))
        T.fill_random(0,1)
        omega = createOmega(I,J,K,sparsity)
        T.i("ijk") << omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
        omega = updateOmega(T,I,J,K)
        
        print("U: ",U)
                
        # purturb the first factor matrix
        U += crandom.random((I,r))*.01
        # call updateU function
        nU = updateU(T,U,V,W,regParam,omega,I,J,K,r)
        
        print("nU: ",nU)
        
        nT = ctf.tensor((I,J,K))
        nT.i("ijk") << omega.i("ijk")*nU.i("iu")*V.i("ju")*W.i("ku")
        
        print("nT: ",nT)
        print("T: ",T)
    
        assert(ctf.all(ctf.abs(nT - T < 1e-10)))
        print("passed test: test_3d_purturb1")

        
    def runAllTests(self):
        self.test_3d_purturb1()

In [3]:
def normalize(Z,r):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [105]:
def createOmega(I,J,K,sparsity):
    Actf = ctf.tensor((I,J,K),sp=True)
    Actf.fill_sp_random(0,1,sparsity)
    omegactf = ((Actf > 0)*ctf.astensor(1.))
    return omegactf


def updateOmega(T,I,J,K):
    '''
    Gets a random subset of rows for each U,V,W iteration
    '''
    omegactf = ((T > 0)*ctf.astensor(1.))
    return omegactf



def getDenseOmega(T,U,V,W,regParam,omega,I,J,K,r,idx,string):
    if (string =="i"):
        omega_curr = ctf.to_nparray(omega[idx,:,:].reshape(J*K))
        omega_sum = np.cumsum(omega_curr)-1
        #print("omega prefix sum: ", omega_sum)
        l = []
        for x,y in enumerate(omega_sum):
            if omega_curr[x] != 0:
                l.append((x,int(y)))
        #print(l)
        num_nonzero = len(l)
        
        # form dense omega matrix
        temp = np.zeros((J*K,len(l)))
        for x,y in l:
            temp[x][y] = 1
            omega_dense = temp.T
        #print("omega_dense: ", omega_dense)
    
        omega_dense = ctf.astensor(omega_dense)
        omega_dense = omega_dense.reshape(J,K,num_nonzero)
        
    
    if (string =="j"):
        omega_curr = ctf.to_nparray(omega[:,idx,:].reshape(I*K))
        omega_sum = np.cumsum(omega_curr)-1
        l = []
        for x,y in enumerate(omega_sum):
            if omega_curr[x] != 0:
                l.append((x,int(y)))
        num_nonzero = len(l)
        # form dense omega matrix
        temp = np.zeros((I*K,len(l)))
        for x,y in l:
            temp[x][y] = 1
            omega_dense = temp.T  
        omega_dense = ctf.astensor(omega_dense)
        omega_dense = omega_dense.reshape(I,K,num_nonzero)        
    
    if (string =="k"):
        omega_curr = ctf.to_nparray(omega[:,:,idx].reshape(I*J))
        omega_sum = np.cumsum(omega_curr)-1
        l = []
        for x,y in enumerate(omega_sum):
            if omega_curr[x] != 0:
                l.append((x,int(y)))
        num_nonzero = len(l)  
        # form dense omega matrix
        temp = np.zeros((I*J,len(l)))
        for x,y in l:
            temp[x][y] = 1
            omega_dense = temp.T   
        omega_dense = ctf.astensor(omega_dense)
        omega_dense = omega_dense.reshape(I,J,num_nonzero)
        
    return num_nonzero,omega_dense

In [106]:
def LS_SVD(A,factor,r,Tbar):
    [U_,S_,V_] = ctf.svd(A)
    S_ = S_/(S_*S_ + regParam*ctf.ones(r))
    
    factor.set_zero()
    factor.i("ir") << V_.i("kr")*S_.i("k")*U_.i("tk")*Tbar.i("it")
    
    return factor    

In [107]:
def updateU(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update U matrix by using the formula'''
    M1 = ctf.tensor((J,K,r))
    M1.i("jku") << V.i("ju")*W.i("ku")
    
    for i in range(I):
        num_nonzero, dense_omega = getDenseOmega(T,U,V,W,regParam,omega,I,J,K,r,i,"i")
        Z = ctf.tensor((num_nonzero,r))
        Z.i("tr") << dense_omega.i("jkt")*M1.i("jkr")
        
        Tbar = ctf.tensor((I,num_nonzero))
        Tbar.i("it") << T.i("ijk")*dense_omega.i("jkt")
        
        U = LS_SVD(Z,U,r,Tbar) 
    #U *= normalize(U,r)
     
    return U
    
    
def updateV(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M2 = ctf.tensor((I,K,r))
    M2.i("iku") << U.i("iu")*W.i("ku")
    
    for j in range(J):
        num_nonzero, dense_omega = getDenseOmega(T,U,V,W,regParam,omega,I,J,K,r,j,"j")
        Z = ctf.tensor((num_nonzero,r))
        Z.i("tr") << dense_omega.i("ikt")*M1.i("ikr")
        
        Tbar = ctf.tensor((J,num_nonzero))
        Tbar.i("jt") << T.i("ijk")*dense_omega.i("ikt")
        
        V = LS_SVD(Z,V,r,Tbar)
    #V *= normalize(V,r)
    
    return V  

def updateW(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M3 = ctf.tensor((I,J,r))
    M3.i("iju") << U.i("iu")*V.i("ju")
    
    for k in range(K):
        num_nonzero, dense_omega = getDenseOmega(T,U,V,W,regParam,omega,I,J,K,r,k,"k")
        Z = ctf.tensor((num_nonzero,r))
        Z.i("tr") << dense_omega.i("ijt")*M1.i("ijr")
        
        Tbar = ctf.tensor((K,num_nonzero))
        Tbar.i("kt") << T.i("ijk")*dense_omega.i("ijt")
        
        W = LS_SVD(Z,W,r,Tbar)
    #W *= normalize(W,r)
    
    return W

In [108]:
def getALSCtf(T,U,V,W,regParam,omega,I,J,K,r):
    '''
    Same thing as above, but CTF
    '''
    it = 0
    E = ctf.tensor((I,J,K))
    E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
    curr_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
    
    while True:
        U = updateU(T,U,V,W,regParam,omega,I,J,K,r)
        V = updateV(T,U,V,W,regParam,omega,I,J,K,r) 
        W = updateW(T,U,V,W,regParam,omega,I,J,K,r)
        E.set_zero()
        E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
        next_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
        
        if abs(curr_err_norm - next_err_norm) < .001 or it > 100:
            break
            
        print(curr_err_norm, next_err_norm)
        #print(next_err_norm/curr_err_norm)
        curr_err_norm = next_err_norm
        it += 1
    
    print("Number of iterations: ", it)
    return U,V,W

In [109]:
def main():
    
    #ut = UnitTests()
    #ut.runAllTests()

    I = random.randint(10,20)
    J = random.randint(10,20)
    K = random.randint(10,20)
    r = 2 
    sparsity = .1
    regParam = 0.001
        
    ctf.random.seed(42)
    U = ctf.random.random((I,r))
    V= ctf.random.random((J,r))
    W= ctf.random.random((K,r))
    
    # 3rd-order tensor
    T = ctf.tensor((I,J,K),sp=True)
    T.fill_sp_random(0,1,sparsity)
    omega = updateOmega(T,I,J,K)
    
    U = ctf.random.random((I,r))
    V= ctf.random.random((J,r))
    W= ctf.random.random((K,r))
    
    t = time.time()
    
    getALSCtf(T,U,V,W,regParam,omega,I,J,K,r)
    
    print("ALS costs time = ",np.round_(time.time()- t,4))    

In [110]:
main()

6.008075310514884 8.232731907777827
8.232731907777827 8.231538380690708
Number of iterations:  2
ALS costs time =  4.4882


## TEST

In [50]:
I = 3
J = 4
K = 5
r = 2
sparsity = .1
regParam = 0.001
        
ctf.random.seed(42)
U = ctf.random.random((I,r))
V= ctf.random.random((J,r))
W= ctf.random.random((K,r))
    
# 3rd-order tensor
T = ctf.tensor((I,J,K),sp=True)
T.fill_sp_random(0,1,sparsity)
omega = updateOmega(T,I,J,K)
T

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.74516551,  0.        ,  0.32375825,  0.75096938,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.80433241,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.04351748,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.92610249,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]])

In [93]:
num_nonzero,dense_omega = getDenseOmega(T,U,V,W,regParam,omega,I,J,K,r,0,'i')
dense_omega.shape

(4, 5, 3)

In [94]:
M1 = ctf.tensor((J,K,r))
M1.i("jku") << V.i("ju")*W.i("ku")
Z = ctf.tensor((num_nonzero,r))
Z.i("tr") << dense_omega.i("jkt")*M1.i("jkr")
print("Z: ",Z)
Z.shape, dense_omega.shape, M1.shape

Z:  array([[ 0.63262532,  0.32515675],
       [ 0.        ,  0.        ],
       [ 0.43277699,  0.16739528]])


((3, 2), (4, 5, 3), (4, 5, 2))

In [96]:
Tbar = ctf.tensor((I,num_nonzero))
Tbar.i("it") << T.i("ijk")*dense_omega.i("jkt")
[U_,S_,V_] = ctf.svd(Z)
S_ = S_/(S_*S_ + regParam*ctf.ones(r))
U.set_zero()
U.i("ir") << V_.i("rk")*S_.i("k")*U_.i("tk")*Tbar.i("it")

In [ ]:
l = np.array([1,1,0,0,1,0,1,0,1])
cuml = np.cumsum(l)-1
result = []
for x,y in enumerate(cuml):
    if l[x] != 0:
        result.append((x,y))
result

In [ ]:
m = np.zeros((result[len(result)-1][0]+1, result[len(result)-1][1]+1))
for x,y in result:
    m[x][y] = 1
    dense = m.T
dense

In [ ]:
temp = ctf.to_nparray(omega[0,:,:].reshape(J*K))
temp

In [ ]:
np.count_nonzero(ctf.to_nparray(omega[0,:,:].reshape(J*K)))